<a href="https://colab.research.google.com/github/GGMmattos/TCC/blob/main/TCC_Gabriel_Matos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libs

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from google.colab import files
from sklearn.preprocessing import StandardScaler

# Importação do dataset

**2018-Fonseca et al**

* Automatically Grading Brazilian Student Essays - NILC Metrix  - [link NILC](http://www.nilc.icmc.usp.br/nilc/projects/unitex-pb/web/dicionarios.html) -  https://github.com/nilc-nlp/nilcmetrix



In [ ]:
# Install huggingface_hub to inspect the repository
# !pip install huggingface_hub -q
# !huggingface-cli login
# import pandas as pd
# import numpy as np
# import torch
# from transformers import AutoTokenizer  # Or BertTokenizer
# from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
# from transformers import AutoModel  # or BertModel, for BERT without pretraining heads

Importação do dataset [Link](https://huggingface.co/datasets/kamel-usp/aes_enem_dataset)

In [ ]:
# splits = {'train': 'JBCS2025/train-00000-of-00001.parquet', 'validation': 'JBCS2025/validation-00000-of-00001.parquet', 'test': 'JBCS2025/test-00000-of-00001.parquet'}
# df_JBCS2025 = pd.read_parquet("hf://datasets/kamel-usp/aes_enem_dataset/" + splits["train"])

In [ ]:
# # Usado no trabalho de referência Silveira (2024)
# splits = {'train': 'PROPOR2024/train-00000-of-00001.parquet', 'validation': 'PROPOR2024/validation-00000-of-00001.parquet', 'test': 'PROPOR2024/test-00000-of-00001.parquet'}
# df_PROPOR2024 = pd.read_parquet("hf://datasets/kamel-usp/aes_enem_dataset/" + splits["train"])

In [ ]:
# splits = {'train': 'gradesThousand/train-00000-of-00001.parquet', 'validation': 'gradesThousand/validation-00000-of-00001.parquet', 'test': 'gradesThousand/test-00000-of-00001.parquet'}
# df_gradesThousand = pd.read_parquet("hf://datasets/kamel-usp/aes_enem_dataset/" + splits["train"])

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame_Final/DataFrame_Final-1.0.csv')


In [ ]:
df.head()

,id_texto,essay_text,grades,adjective_ratio,adverbs,content_words,flesch,function_words,sentences_per_paragraph,syllables_per_content_word,...,sentence_length_min,sentence_length_standard_deviation,short_sentence_ratio,std_noun_phrase,verb_diversity,verbs_max,verbs_min,verbs_standard_deviation,long_sentence_ratio,ratio_function_to_content_words
0,redacao_1,"Ultimamente, temos observado, um aumento consi...",[0 0 0 0 0 0],5.495,5.495,54.396,1.201.548,45.604,1.500,272.727,...,9,5.038.739,33.333,236.388,9.375,18.182,11.111,2.987,66.667,83.838
1,redacao_2,"Infelizmente, no Brasil, na maioria dos lugare...",[0 0 0 0 0 0],7.107,3.553,58.883,3.488.631,41.117,2.000,291.379,...,9,1.126.873,125.000,302.367,85.714,23.529,11.111,387.000,625.000,69.828
2,redacao_3,"Em todos os lugares do Brasil temos violência,...",[0 0 0 0 0 0],2.564,5.128,53.846,4.271.548,46.154,133.333,279.365,...,13,1.166.994,0.000,403.856,83.333,21.739,15.385,2.564,0.750,85.714
3,redacao_4,"No Brasil, o número de cidadãos que querem mig...",[0 0 0 0 0 0],3.571,5.952,61.111,43.783,38.889,3.000,283.117,...,4,576.426,0.200,3.812,86.047,33.333,5.882,7.976,66.667,63.636
4,redacao_5,"Com apenas 20 anos de idade, o poeta romântico...",[ 200 200 200 200 200 1000],7.459,663.000,61.602,327.519,38.398,4.000,304.036,...,6,730.685,0.250,379.162,92.157,33.333,0.000,6.721,0.700,62.332


# Gerando embedding

BERTimbau (como a maioria dos BERTs base) tem um limite de 512 tokens para a entrada. Redações do ENEM geralmente são mais longas.

**Código para obter embedding (truncamento)** OBS: não utilizado para o nosso trabalho



Simplesmente cortar a redação após 512 tokens. Simples, mas pode perder informações críticas.

In [ ]:
# def get_bert_embedding(text, tokenizer, model, device, max_length=512):
#     inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=max_length, padding='max_length')
#     inputs = {k: v.to(device) for k, v in inputs.items()}

#     with torch.no_grad(): # Desativa o cálculo de gradientes para inferência, economiza memória
#         outputs = model(**inputs)

#     # Retorna o embedding do token [CLS] da última camada
#     # outputs.last_hidden_state[0, 0, :] é o embedding do [CLS]
#     return outputs.last_hidden_state[0, 0, :].cpu().numpy()

**Código para obter embedding (segmentação)**

Dividir a redação em pedaços de 512 tokens, gerar um embedding para cada pedaço e depois combiná-los

In [ ]:
def get_bert_embedding_segmented(text, tokenizer, model, device, max_length=512, stride=256):
    """
    Gera um embedding BERT para um texto longo, utilizando segmentação e agregação.

    Args:
        text (str): O texto da redação.
        tokenizer: O tokenizer do modelo BERT.
        model: O modelo BERT (ex: BERTimbau).
        device (torch.device): 'cuda' ou 'cpu'.
        max_length (int): O tamanho máximo de cada segmento para o BERT (default: 512).
        stride (int): O tamanho da sobreposição entre os segmentos (default: 256).
                      Um stride menor significa mais sobreposição e possivelmente
                      maior captura de contexto, mas mais processamento.

    Returns:
        np.array: O embedding combinado (média) de todos os segmentos, ou NaNs se o texto for nulo.
    """
    if pd.isna(text) or not isinstance(text, str): # Lidar com possíveis valores NaN ou não-string
        return np.full((model.config.hidden_size,), np.nan) # Retorna um array de NaNs

    # Tokenizar o texto completo com o retorno de IDs (para controlar a segmentação)
    # add_special_tokens=False para não adicionar [CLS]/[SEP] no começo e fim de cada segmento
    # pois queremos controlar isso.
    token_ids = tokenizer.encode(text, add_special_tokens=False)

    # Lista para armazenar os embeddings de cada segmento
    segment_embeddings = []

    # Iterar sobre os tokens para criar segmentos
    # max_length - 2 para dar espaço para [CLS] e [SEP] que serão adicionados pelo tokenizer em cada segmento
    effective_max_length = max_length - 2

    # Se o texto for muito curto, trata como um único segmento (mesmo que menor que max_length)
    if len(token_ids) <= effective_max_length:
        segments = [token_ids]
    else:
        segments = []
        for i in range(0, len(token_ids), effective_max_length - stride):
            segment = token_ids[i : i + effective_max_length]
            segments.append(segment)
            # Se o último segmento for menor que o stride (não suficiente para sobreposição),
            # ou se já chegamos ao final, paramos.
            if i + effective_max_length >= len(token_ids):
                break


    # Processar cada segmento
    for segment in segments:
        # Adicionar os tokens especiais para cada segmento individualmente
        # return_tensors='pt' para PyTorch
        # truncation=True é redundante aqui se já controlamos o tamanho, mas é bom manter
        # padding='max_length' garante que todos os segmentos tenham o mesmo tamanho de input
        inputs = tokenizer.prepare_for_model(
            segment,
            add_special_tokens=True,
            return_tensors='pt',
            max_length=max_length,
            padding='max_length',
            truncation=True # Em caso de erro na lógica de segmentação, isso garante
        )

        # Add batch dimension to each tensor in the inputs dictionary
        inputs = {k: v.unsqueeze(0).to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)

        # Retorna o embedding do token [CLS] do segmento
        # outputs.last_hidden_state will now have shape (batch_size, sequence_length, hidden_size)
        # We still want the CLS token for the single item in the batch, which is index 0
        segment_embedding = outputs.last_hidden_state[0, 0, :].cpu().numpy()
        segment_embeddings.append(segment_embedding)

    if not segment_embeddings: # Se por algum motivo nenhum embedding foi gerado (ex: texto vazio após pré-processamento)
        return np.full((model.config.hidden_size,), np.nan)

    # Combinar os embeddings dos segmentos pela média
    combined_embedding = np.mean(segment_embeddings, axis=0)

    return combined_embedding

In [ ]:
# 1. Carregar o Tokenizer e o Modelo BERTimbau
tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")
model = AutoModel.from_pretrained("neuralmind/bert-base-portuguese-cased")

# Mover o modelo para a GPU, se disponível
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval() # Coloca o modelo em modo de avaliação

In [ ]:
df['bert_embedding'] = df['essay_text'].apply(
    lambda text: get_bert_embedding_segmented(text, tokenizer, model, device, max_length=512, stride=256)
)

print("\nDataFrame com embeddings BERTimbau segmentados:")
print(df.head())

# Para verificar o formato do primeiro embedding:
print(f"\nShape do primeiro embedding segmentado: {df['bert_embedding'].iloc[0].shape}")

In [ ]:
df.head()

,id_texto,essay_text,grades,adjective_ratio,adverbs,content_words,flesch,function_words,sentences_per_paragraph,syllables_per_content_word,...,sentence_length_standard_deviation,short_sentence_ratio,std_noun_phrase,verb_diversity,verbs_max,verbs_min,verbs_standard_deviation,long_sentence_ratio,ratio_function_to_content_words,bert_embedding
0,redacao_1,"Ultimamente, temos observado, um aumento consi...",[0 0 0 0 0 0],5.495,5.495,54.396,1.201.548,45.604,1.500,272.727,...,5.038.739,33.333,236.388,9.375,18.182,11.111,2.987,66.667,83.838,"[-0.08385735, 0.17223401, -0.44074976, -0.0477..."
1,redacao_2,"Infelizmente, no Brasil, na maioria dos lugare...",[0 0 0 0 0 0],7.107,3.553,58.883,3.488.631,41.117,2.000,291.379,...,1.126.873,125.000,302.367,85.714,23.529,11.111,387.000,625.000,69.828,"[-0.16432929, -0.091625646, 0.059742924, -0.32..."
2,redacao_3,"Em todos os lugares do Brasil temos violência,...",[0 0 0 0 0 0],2.564,5.128,53.846,4.271.548,46.154,133.333,279.365,...,1.166.994,0.000,403.856,83.333,21.739,15.385,2.564,0.750,85.714,"[-0.058431715, 0.22637141, -0.286155, 0.034034..."
3,redacao_4,"No Brasil, o número de cidadãos que querem mig...",[0 0 0 0 0 0],3.571,5.952,61.111,43.783,38.889,3.000,283.117,...,576.426,0.200,3.812,86.047,33.333,5.882,7.976,66.667,63.636,"[-0.26524943, -0.15599328, 0.33433747, -0.5228..."
4,redacao_5,"Com apenas 20 anos de idade, o poeta romântico...",[ 200 200 200 200 200 1000],7.459,663.000,61.602,327.519,38.398,4.000,304.036,...,730.685,0.250,379.162,92.157,33.333,0.000,6.721,0.700,62.332,"[-0.114247955, 0.23296513, -0.013842106, -0.65..."


In [ ]:
# nome_do_arquivo_csv = 'DataFrame_Final.csv'
# df.to_csv(nome_do_arquivo_csv, index=False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/TCC -  Bacharel  Informática/Arquivos desenvolvimento/Base de dados/DataFrame_Final/DataFrame_Final-2.0.csv')


In [ ]:
# Criação da matrix X de features (contém as métricas e os embedding das redações)
X = df.loc[:, 'adjective_ratio':'ratio_function_to_content_words']

# Definição da variável alvo, as notas por comportência
y = df['grades'].values

In [ ]:
# Verifique os dtypes ANTES da limpeza para ver as 'object' (strings)
print("Tipos de dados ANTES da limpeza:")
print(X.dtypes.head(10)) # Mostra as primeiras colunas
print("\n---")

Tipos de dados ANTES da limpeza:
adjective_ratio               float64
adverbs                       float64
content_words                 float64
flesch                         object
function_words                float64
sentences_per_paragraph       float64
syllables_per_content_word    float64
words_per_sentence             object
noun_ratio                    float64
paragraphs                      int64
dtype: object

---


In [ ]:
colunas_para_limpar = [col for col in X.columns] # pegamos todas as métricas

In [ ]:
# 2. Iterar sobre as colunas e limpar/converter
for col in colunas_para_limpar:
    # APLICAR A LIMPEZA SOMENTE SE A COLUNA FOR DE TIPO 'object' (string)
    if X[col].dtype == 'object':
        # Substitui o ponto (separador de milhar) por vazio
        # E substitui a vírgula (separador decimal, se houvesse) por ponto
        # Depois, converte para numérico
        # errors='coerce' converte qualquer valor que não possa ser numérico para NaN
        X[col] = X[col].astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
        X[col] = pd.to_numeric(X[col], errors='coerce') # Use pd.to_numeric para conversão robusta

# Verificar os dtypes DEPOIS da limpeza
print("Tipos de dados DEPOIS da limpeza:")
print(X.dtypes.head(10))
print("\n---")

Tipos de dados DEPOIS da limpeza:
adjective_ratio               float64
adverbs                       float64
content_words                 float64
flesch                          int64
function_words                float64
sentences_per_paragraph       float64
syllables_per_content_word    float64
words_per_sentence              int64
noun_ratio                    float64
paragraphs                      int64
dtype: object

---


Procedimento para não gerar erro ao tentar utilizar o **StandardScaler**

- Isso evita que features com valores maiores dominem o processo de aprendizado.

In [ ]:
# Converter a string para array NumPy
def string_to_array(embedding_str):
    # Remove colchetes e espaços extras, depois divide pelos espaços
    embedding_str = embedding_str.strip('[]')
    # Divide pelos espaços e quebras de linha, filtrando valores vazios
    numbers = [float(x) for x in embedding_str.replace('\n', ' ').split() if x]
    return np.array(numbers)

# Aplica a conversão para toda a coluna
conversao_embedding = df['bert_embedding'].apply(string_to_array)

In [ ]:
X_metrics  = np.vstack(X.values)
X_embeddings = np.vstack(conversao_embedding.values)
X_combined = np.hstack((X_metrics, X_embeddings))

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_combined)

In [ ]:
X_scaled

array([[-0.31313934, -0.29857313, -0.04225708, ...,  0.67457778,
         0.89087485,  0.23639946],
       [-0.30542534, -0.31100873,  0.05093573, ..., -0.14111205,
        -0.55805593,  0.65394979],
       [-0.32716521, -0.30092322, -0.05368032, ...,  0.66601102,
         0.32842944,  0.81986912],
       ...,
       [-0.29351459,  2.25325325,  0.0972102 , ...,  1.13492047,
        -0.31595579, -0.23720102],
       [-0.30221437, -0.28751429, -0.00663737, ...,  0.96717289,
        -0.64077216, -0.35923914],
       [-0.30557847, -0.29449411,  0.003602  , ...,  0.91432766,
         0.46651283, -0.3281742 ]])

#Configurações e ajuste no modelo BERTimbau

**Bert** [Documentação](https://huggingface.co/docs/transformers/model_doc/bert)